MK užduotis 1-5
1. Sugeneruokite pseudoatsitiktinių skaičių sekas tiesiniu kongruentiniu metodu su maksimaliu periodu, kai modulis $m=776$ ir $m=1107$. Daugiklius $a$ parinkite taip, kad galingumai būtų didžiausi. Prieauglio $c$ parinkimui naudokitės gretimų narių koreliacija (teoriniai testai).

Tiesinio kongruentinio metodo formulė:
$X_{n+1} \equiv (aX_n + c) \pmod m, n \geq 0$.

Čia skaičiai:
$X_0$ - pradinė reikšmė, $X_0 \geq 0$,
$a$ - daugiklis, $a \geq 0$,
$c$ - prieauglis, $c \geq 0$,
$m$ - modulis, $m > X_0, m > a, m > c$, yra perenkami.

Gretimų narių koeficientas apibrežiamas taip:
$$
C := \frac{n \sum_{i=0}^{n-1} (U_i V_i) - \left(\sum_{i=0}^{n-1} U_i\right) \left(\sum_{i=0}^{n-1} V_i\right)}{\sqrt{\left(n \sum_{i=0}^{n-1} U_i^2 - \left(\sum_{i=0}^{n-1} U_i\right)^2\right) \left(n \sum_{i=0}^{n-1} V_i^2 - \left(\sum_{i=0}^{n-1} V_i\right)^2\right)}}
$$

Tiesinės kongruentinės sekos $(X_0, a, c, m)$ su maksimaliu periodu gretimų narių koreliacijos koeficientas

$$C \approx \frac{1}{a} \left(1 - 6 \frac{c}{m} + 6 \left(\frac{c}{m}\right)^2\right).$$
Šios apytikslės formulės klaida yra nedidesnė už $(a+6)/m$.

Kad tiesinis kongruentinis generatorius turėtų maksimalų periodą lygų $m$, turi būti tenkinamos šios sąlygos:

Skaičiai $c$ ir $m$ turi būti tarpusavyje pirminiai (jų DBD = 1).
Jei $p$ yra bet kuris pirminis skaičius, kuris dalijasi iš $m$, tada $(a-1)$ turi dalintis iš $p$.
Jei $m$ yra dalus iš 4, tada $(a-1)$ taip pat turi būti dalus iš 4.

Modulis $776 = 8 \times 97 = 2^3 \times 97$

Modulis $1107 = 3 \times 369 = 3^3 \times 41$


In [73]:
def prime_factorization(n):
    # Skaičiaus n pirminių daugiklių radimas
    factors = {}
    d = 2
    while d*d <= n:
        while n % d == 0:
            if d in factors:
                factors[d] += 1
            else:
                factors[d] = 1
            n //= d
        d += 1
    if n > 1:
        if n in factors:
            factors[n] += 1
        else:
            factors[n] = 1
    return factors

print(f"Modulio 776 = {776} pirminiai daugikliai: {prime_factorization(776)}")
print(f"Modulio 1107 = {1107} pirminiai daugikliai: {prime_factorization(1107)}")

Modulio 776 = 776 pirminiai daugikliai: {2: 3, 97: 1}
Modulio 1107 = 1107 pirminiai daugikliai: {3: 3, 41: 1}


In [74]:
import math
from math import gcd
import numpy as np
import scipy.stats as stats

def find_power(a, m):
    """Rasti sekos galią (mažiausią s, kur (a-1)^s ≡ 0 mod m)."""
    b = a - 1
    s = 1
    result = b % m
    
    # Maksimalus iteracijų skaičius, kad būtų išvengta begalinio ciklo
    max_iterations = m
    iterations = 0
    
    while result != 0 and iterations < max_iterations:
        result = (result * b) % m
        s += 1
        iterations += 1
    
    # Jei neradome galios, kuri tenkina b^s ≡ 0 (mod m)
    if result != 0:
        return 0
    
    return s



In [75]:
m_776=776

valid_a_values_776 = []

for a in range(2, m_776):
        # Tikrinti tik reikšmes, kur gcd(a,m) = 1
        if gcd(a, m_776) == 1:
            b = a - 1            # Patikrinti, ar b tenkina mūsų sąlygas
            # Kai m = 776 = 2^3 * 97:
            # b turėtų dalintis iš 2 ir 97 ir 4 (nes 776 % 4 = 0)
            if b % 2 == 0 and b % 97 == 0 and b % 4 == 0:
                power = find_power(a, m_776)
                valid_a_values_776.append((a, b, power))
# Rūšiuoti pagal galią (didesnė yra geresnė)
valid_a_values_776.sort(key=lambda x: x[2], reverse=True)

# Spausdinti rezultatus
print(f"\n{'Daugiklis a':<15}{'b=a-1':<15}{'Galingumas s':<15}")

for a, b, power in valid_a_values_776[:3]:  # Rodyti 3 geriausius rezultatus
    print(f"{a:<15}{b:<15}{power:<15}")

m_1107=1107

valid_a_values_1107 = []

for a in range(2, m_1107):
        # Tikrinti tik reikšmes, kur gcd(a,m) = 1
        if gcd(a, m_1107) == 1:
            b = a - 1
            # Patikrinti, ar b tenkina mūsų sąlygas
            # Kai m = 1107 = 3^3 * 41:
            # b turėtų dalintis iš 3 ir 41
            if b % 3 == 0 and b % 41 == 0:
                power = find_power(a, m_1107)
                valid_a_values_1107.append((a, b, power))
# Rūšiuoti pagal galią (didesnė yra geresnė)
valid_a_values_1107.sort(key=lambda x: x[2], reverse=True)

# Spausdinti rezultatus
print(f"\n{'Daugiklis a':<15}{'b=a-1':<15}{'Galingumas s':<15}")

for a, b, power in valid_a_values_1107[:3]:  # Rodyti 3 geriausius rezultatus
    print(f"{a:<15}{b:<15}{power:<15}")


Daugiklis a    b=a-1          Galingumas s   
389            388            2              

Daugiklis a    b=a-1          Galingumas s   
124            123            3              
247            246            3              
493            492            3              


In [76]:
# Geriausias rezultatas
best_a_776, best_b_776, best_power_776 = valid_a_values_776[0]
print("\nGeriausias rezultatas:")
print(f"Daugiklis a = {best_a_776}")
print(f"b = a - 1 = {best_b_776}")
print(f"Galingumas s = {best_power_776}")

best_a_1107, best_b_1107, best_power_1107 = valid_a_values_1107[0]
print("\nGeriausias rezultatas:")
print(f"Daugiklis a = {best_a_1107}")
print(f"b = a - 1 = {best_b_1107}")
print(f"Galingumas s = {best_power_1107}")


Geriausias rezultatas:
Daugiklis a = 389
b = a - 1 = 388
Galingumas s = 2

Geriausias rezultatas:
Daugiklis a = 124
b = a - 1 = 123
Galingumas s = 3


In [77]:
def find_valid_c(m):
    valid_c_values = []
    
    for c in range(1, m):
        if gcd(c, m) == 1:
            # Jei m dalijasi iš 4, c turi būti nelyginis
            if m % 4 == 0 and c % 2 == 0:
                continue
            valid_c_values.append(c)
    
    return valid_c_values


In [78]:
print(f"b^s mod m = {best_b_776}^{best_power_776} mod {m_776} = {pow(best_b_776, best_power_776, m_776)}")
# Rasti tinkamas c reikšmes
valid_c_values = find_valid_c(m_776)


print(f"b^s mod m = {best_b_1107}^{best_power_1107} mod {m_1107} = {pow(best_b_1107, best_power_1107, m_1107)}")
# Rasti tinkamas c reikšmes
valid_c_values = find_valid_c(m_1107)


b^s mod m = 388^2 mod 776 = 0
b^s mod m = 123^3 mod 1107 = 0


In [79]:

def calculate_theoretical_correlation(a, c, m):
    """
    C ≈ (1/a) * (1 - 6(c/m) + 6(c/m)²)
    
    Paklaidos formulė (a+6)/m
    """
    c_m_ratio = c / m
    correlation = (1/a) * (1 - 6 * c_m_ratio + 6 * (c_m_ratio ** 2))
    error_bound = (a + 6) / m
    
    return abs(correlation), error_bound

def test_c_correlation(a, m, valid_c_values, num_tests):
    """Calculate theoretical correlation for different c values."""
    c_theoretical_correlations = []
    
    for c in valid_c_values[:num_tests]:
        # Only theoretical correlation
        theoretical_correlation, error_bound = calculate_theoretical_correlation(a, c, m)
        c_theoretical_correlations.append((c, theoretical_correlation, error_bound))
    
    # Sort by theoretical correlation (lower is better)
    c_theoretical_correlations.sort(key=lambda x: x[1])
    
    return c_theoretical_correlations

In [80]:
theoretical_correlations_776 = test_c_correlation(best_a_776, m_776, valid_c_values, num_tests=1000)

theoretical_correlations_1107 = test_c_correlation(best_a_1107, m_1107, valid_c_values, num_tests=1000)